# Week 6 - Interactive Visualizations Part 3: Advanced Interactive Features

## Learning Objectives
By the end of this lesson, you will be able to:
1. Implement advanced interactive controls (dropdowns, sliders, buttons)
2. Create cross-filtering dashboards where charts interact with each other
3. Build animated visualizations for storytelling
4. Design responsive layouts that adapt to different screen sizes
5. Apply performance optimization techniques for large datasets

## Business Context: Next-Level Interactivity

In today's competitive business environment, static dashboards are no longer sufficient. Modern business intelligence requires **self-service analytics** where stakeholders can:

- **Explore data dynamically** - Filter, drill-down, and discover insights independently
- **Tell data stories** - Use animation and progressive disclosure to communicate findings
- **Adapt to context** - Dashboards that work on desktop, tablet, and mobile devices
- **Handle real-time data** - Performance-optimized visualizations for live business metrics

Today we'll master these advanced techniques using real Olist e-commerce data to create truly interactive business intelligence tools.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import zipfile
import io
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Advanced styling configuration
ADVANCED_THEME = {
    'colors': {
        'primary': '#2E86AB',      # Deep blue
        'secondary': '#A23B72',    # Burgundy
        'accent': '#F18F01',       # Orange
        'success': '#C73E1D',      # Red
        'info': '#8E44AD',         # Purple
        'warning': '#F39C12',      # Yellow
        'light': '#ECF0F1',        # Light gray
        'dark': '#2C3E50'          # Dark blue-gray
    },
    'fonts': {
        'title': {'family': 'Arial, sans-serif', 'size': 18, 'color': '#2C3E50'},
        'body': {'family': 'Arial, sans-serif', 'size': 12, 'color': '#34495E'},
        'axis': {'family': 'Arial, sans-serif', 'size': 11, 'color': '#7F8C8D'}
    }
}

print("🚀 Advanced Interactive Visualization Environment Ready!")
print("Features: Widget Controls, Animation, Cross-filtering, Performance Optimization")

In [ ]:
# Load and prepare enhanced dataset for advanced interactivity
def load_enhanced_olist_data():
    """
    Load Olist data with enhanced features for advanced interactivity.
    """
    try:
        zip_url = "https://github.com/autom8or-com/python-data-analysis-course/raw/main/Resources/data/sales.zip"
        
        print("Loading enhanced Olist dataset...")
        response = requests.get(zip_url)
        response.raise_for_status()
        
        datasets = {}
        
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            file_mapping = {
                'customers': 'olist_customers_dataset.csv',
                'orders': 'olist_orders_dataset.csv',
                'order_items': 'olist_order_items_dataset.csv',
                'products': 'olist_products_dataset.csv',
                'sellers': 'olist_sellers_dataset.csv',
                'payments': 'olist_order_payments_dataset.csv',
                'reviews': 'olist_order_reviews_dataset.csv',
                'geolocation': 'olist_geolocation_dataset.csv'
            }
            
            for table_name, filename in file_mapping.items():
                try:
                    datasets[table_name] = pd.read_csv(zip_file.open(filename))
                    print(f"✓ {table_name}: {datasets[table_name].shape[0]:,} rows")
                except KeyError:
                    continue
        
        return datasets
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return create_enhanced_sample_data()

def create_enhanced_sample_data():
    """
    Create enhanced sample data optimized for advanced interactive features.
    """
    np.random.seed(42)
    
    # Generate time-series data with realistic patterns
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2018, 8, 31)
    date_range = pd.date_range(start_date, end_date, freq='D')
    
    # Enhanced customer data
    n_customers = 10000
    brazilian_states = ['SP', 'RJ', 'MG', 'RS', 'PR', 'SC', 'BA', 'GO', 'PE', 'CE', 'DF', 'ES']
    state_weights = [0.35, 0.15, 0.12, 0.08, 0.06, 0.05, 0.04, 0.03, 0.03, 0.03, 0.03, 0.03]
    
    customers = pd.DataFrame({
        'customer_id': [f'cust_{i:06d}' for i in range(n_customers)],
        'customer_state': np.random.choice(brazilian_states, n_customers, p=state_weights),
        'customer_city': np.random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 
                                         'Porto Alegre', 'Curitiba', 'Salvador', 'Brasília', 
                                         'Fortaleza', 'Recife', 'Goiânia'], n_customers)
    })
    
    # Enhanced orders with seasonal patterns
    n_orders = 20000
    
    # Create seasonal multipliers
    order_dates = np.random.choice(date_range, n_orders)
    seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * pd.to_datetime(order_dates).dayofyear / 365.25)
    
    orders = pd.DataFrame({
        'order_id': [f'ord_{i:08d}' for i in range(n_orders)],
        'customer_id': np.random.choice(customers['customer_id'], n_orders),
        'order_status': np.random.choice(['delivered', 'shipped', 'processing', 'canceled'], 
                                       n_orders, p=[0.87, 0.07, 0.04, 0.02]),
        'order_purchase_timestamp': order_dates,
        'order_delivered_customer_date': order_dates + pd.to_timedelta(
            np.random.exponential(7, n_orders), unit='days'
        )
    })
    
    # Enhanced products with realistic categories
    categories = [
        'health_beauty', 'computers_accessories', 'furniture_decor', 'sports_leisure',
        'housewares', 'watches_gifts', 'telephony', 'auto', 'toys', 'cool_stuff',
        'garden_tools', 'perfumery', 'baby', 'electronics', 'stationery', 'books',
        'home_appliances', 'fashion_clothes_shoes', 'jewelry', 'music'
    ]
    
    n_products = 5000
    products = pd.DataFrame({
        'product_id': [f'prod_{i:06d}' for i in range(n_products)],
        'product_category_name': np.random.choice(categories, n_products),
        'product_weight_g': np.random.lognormal(6.5, 1.2, n_products),
        'product_length_cm': np.random.lognormal(3.2, 0.6, n_products),
        'product_height_cm': np.random.lognormal(2.8, 0.5, n_products),
        'product_width_cm': np.random.lognormal(3.0, 0.6, n_products)
    })
    
    # Enhanced order items with realistic pricing patterns
    n_items = 35000
    order_items = pd.DataFrame({
        'order_id': np.random.choice(orders['order_id'], n_items),
        'product_id': np.random.choice(products['product_id'], n_items),
        'seller_id': [f'sell_{i:04d}' for i in np.random.randint(0, 1000, n_items)],
        'price': np.random.lognormal(3.2, 0.9, n_items),
        'freight_value': np.random.exponential(12, n_items)
    })
    
    # Enhanced reviews with realistic distribution
    reviews = pd.DataFrame({
        'order_id': np.random.choice(orders['order_id'], int(n_orders * 0.85)),
        'review_score': np.random.choice([1, 2, 3, 4, 5], int(n_orders * 0.85), 
                                       p=[0.03, 0.05, 0.12, 0.35, 0.45]),
        'review_creation_date': np.random.choice(date_range, int(n_orders * 0.85))
    })
    
    # Enhanced geolocation data
    geolocation = pd.DataFrame({
        'geolocation_zip_code_prefix': np.random.randint(10000, 99999, 1000),
        'geolocation_lat': np.random.uniform(-33.75, 5.27, 1000),  # Brazil's lat range
        'geolocation_lng': np.random.uniform(-73.98, -34.79, 1000),  # Brazil's lng range
        'geolocation_city': np.random.choice(customers['customer_city'].unique(), 1000),
        'geolocation_state': np.random.choice(customers['customer_state'].unique(), 1000)
    })
    
    print("✓ Enhanced sample datasets created with advanced features")
    print(f"  - Time range: {len(date_range)} days")
    print(f"  - Seasonal patterns: Included")
    print(f"  - Geographic data: {len(geolocation)} locations")
    
    return {
        'customers': customers,
        'orders': orders,
        'order_items': order_items,
        'products': products,
        'reviews': reviews,
        'geolocation': geolocation
    }

# Load enhanced data
enhanced_data = load_enhanced_olist_data()
print("\n🎯 Enhanced dataset ready for advanced interactivity!")

In [ ]:
def prepare_advanced_analytics_dataset(data):
    """
    Prepare a comprehensive dataset optimized for advanced interactive analytics.
    """
    print("Preparing advanced analytics dataset...")
    
    # Start with orders as the base
    df = data['orders'].copy()
    
    # Convert dates with proper error handling
    date_columns = ['order_purchase_timestamp', 'order_delivered_customer_date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Add customer information
    if 'customers' in data:
        df = df.merge(data['customers'], on='customer_id', how='left')
    
    # Calculate comprehensive order metrics
    if 'order_items' in data:
        order_financials = data['order_items'].groupby('order_id').agg({
            'price': ['sum', 'mean', 'count'],
            'freight_value': ['sum', 'mean'],
            'product_id': 'nunique',
            'seller_id': 'nunique'
        }).round(2)
        
        order_financials.columns = [
            'order_value', 'avg_item_price', 'items_count',
            'total_freight', 'avg_freight', 'unique_products', 'unique_sellers'
        ]
        order_financials = order_financials.reset_index()
        order_financials['total_value'] = order_financials['order_value'] + order_financials['total_freight']
        
        df = df.merge(order_financials, on='order_id', how='left')
    
    # Add product category information
    if 'products' in data and 'order_items' in data:
        # Get the most expensive item's category per order
        order_categories = data['order_items'].merge(
            data['products'][['product_id', 'product_category_name']], 
            on='product_id', how='left'
        )
        
        primary_categories = order_categories.loc[
            order_categories.groupby('order_id')['price'].idxmax()
        ][['order_id', 'product_category_name']]
        
        df = df.merge(primary_categories, on='order_id', how='left')
    
    # Add review information
    if 'reviews' in data:
        review_metrics = data['reviews'].groupby('order_id').agg({
            'review_score': ['mean', 'count']
        }).round(2)
        review_metrics.columns = ['avg_review_score', 'review_count']
        review_metrics = review_metrics.reset_index()
        
        df = df.merge(review_metrics, on='order_id', how='left')
    
    # Create comprehensive time-based features
    if 'order_purchase_timestamp' in df.columns:
        df['year'] = df['order_purchase_timestamp'].dt.year
        df['month'] = df['order_purchase_timestamp'].dt.month
        df['quarter'] = df['order_purchase_timestamp'].dt.quarter
        df['week'] = df['order_purchase_timestamp'].dt.isocalendar().week
        df['weekday'] = df['order_purchase_timestamp'].dt.day_name()
        df['hour'] = df['order_purchase_timestamp'].dt.hour
        df['day_of_year'] = df['order_purchase_timestamp'].dt.dayofyear
        df['is_weekend'] = df['order_purchase_timestamp'].dt.weekday >= 5
        
        # Create period columns for filtering
        df['year_month'] = df['order_purchase_timestamp'].dt.to_period('M')
        df['year_quarter'] = df['order_purchase_timestamp'].dt.to_period('Q')
        df['date_only'] = df['order_purchase_timestamp'].dt.date
    
    # Calculate delivery and logistics metrics
    if all(col in df.columns for col in ['order_purchase_timestamp', 'order_delivered_customer_date']):
        df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
        df['delivery_speed'] = pd.cut(
            df['delivery_days'], 
            bins=[0, 3, 7, 14, 30, float('inf')], 
            labels=['Same Day', 'Fast', 'Normal', 'Slow', 'Very Slow']
        )
    
    # Create business segments
    if 'total_value' in df.columns:
        df['order_size_segment'] = pd.cut(
            df['total_value'], 
            bins=[0, 50, 150, 300, float('inf')], 
            labels=['Small', 'Medium', 'Large', 'Premium']
        )
    
    if 'avg_review_score' in df.columns:
        df['satisfaction_level'] = pd.cut(
            df['avg_review_score'], 
            bins=[0, 2, 3, 4, 5], 
            labels=['Poor', 'Fair', 'Good', 'Excellent']
        )
    
    # Clean and fill missing values
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)
    
    categorical_fills = {
        'customer_state': 'Unknown',
        'customer_city': 'Unknown',
        'product_category_name': 'uncategorized',
        'delivery_speed': 'Normal',
        'order_size_segment': 'Small',
        'satisfaction_level': 'Good'
    }
    
    for col, fill_value in categorical_fills.items():
        if col in df.columns:
            df[col] = df[col].fillna(fill_value)
    
    print(f"✓ Advanced analytics dataset prepared:")
    print(f"  - {df.shape[0]:,} orders with {df.shape[1]} features")
    print(f"  - Date range: {df['order_purchase_timestamp'].min()} to {df['order_purchase_timestamp'].max()}")
    print(f"  - Business segments: {df['order_size_segment'].nunique()} order sizes, {df['satisfaction_level'].nunique()} satisfaction levels")
    
    return df

# Prepare comprehensive dataset
advanced_df = prepare_advanced_analytics_dataset(enhanced_data)

# Display key metrics
print(f"\n📊 Dataset Overview:")
print(f"  💰 Total Revenue: ${advanced_df['total_value'].sum():,.0f}")
print(f"  📦 Total Orders: {len(advanced_df):,}")
print(f"  👥 Unique Customers: {advanced_df['customer_id'].nunique():,}")
print(f"  🛍️ Product Categories: {advanced_df['product_category_name'].nunique()}")
print(f"  🗺️ States Covered: {advanced_df['customer_state'].nunique()}")

## 1. Widget-Based Interactive Controls

Interactive widgets allow users to dynamically filter and explore data without writing code. This is essential for self-service analytics where business users need to explore data independently.

### Key Widget Types for Business Analytics:
- **Dropdown menus** - Category selection, time period filtering
- **Sliders** - Date ranges, value thresholds
- **Checkboxes** - Multi-selection options
- **Radio buttons** - Exclusive choice options
- **Date pickers** - Time period selection

In [ ]:
def create_interactive_sales_dashboard():
    """
    Create an interactive sales dashboard with widget controls.
    """
    
    def update_dashboard(selected_categories, date_range, order_size, customer_states):
        """
        Update dashboard based on widget selections.
        """
        # Filter data based on selections
        filtered_df = advanced_df.copy()
        
        # Apply filters
        if selected_categories:
            filtered_df = filtered_df[filtered_df['product_category_name'].isin(selected_categories)]
        
        if customer_states:
            filtered_df = filtered_df[filtered_df['customer_state'].isin(customer_states)]
        
        if order_size != 'All':
            filtered_df = filtered_df[filtered_df['order_size_segment'] == order_size]
        
        # Apply date filter
        start_date, end_date = date_range
        filtered_df = filtered_df[
            (filtered_df['order_purchase_timestamp'] >= start_date) & 
            (filtered_df['order_purchase_timestamp'] <= end_date)
        ]
        
        if len(filtered_df) == 0:
            print("⚠️ No data matches the selected filters. Please adjust your selections.")
            return
        
        # Create dashboard with filtered data
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=(
                f'📈 Revenue Trend ({len(filtered_df):,} orders)',
                '🛍️ Top Categories',
                '⭐ Satisfaction Distribution',
                '🗺️ Geographic Performance',
                '📊 Order Size Distribution',
                '🚚 Delivery Performance'
            ),
            specs=[
                [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}],
                [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]
            ],
            vertical_spacing=0.12,
            horizontal_spacing=0.08
        )
        
        # Chart 1: Revenue trend
        daily_revenue = filtered_df.groupby('date_only')['total_value'].sum().reset_index()
        
        fig.add_trace(
            go.Scatter(
                x=daily_revenue['date_only'],
                y=daily_revenue['total_value'],
                mode='lines',
                name='Daily Revenue',
                line=dict(color=ADVANCED_THEME['colors']['primary'], width=2),
                fill='tonexty'
            ),
            row=1, col=1
        )
        
        # Chart 2: Top categories
        category_revenue = filtered_df.groupby('product_category_name')['total_value'].sum().nlargest(8)
        
        fig.add_trace(
            go.Bar(
                x=category_revenue.values,
                y=[cat.replace('_', ' ').title() for cat in category_revenue.index],
                orientation='h',
                marker_color=ADVANCED_THEME['colors']['secondary'],
                name='Category Revenue'
            ),
            row=1, col=2
        )
        
        # Chart 3: Satisfaction distribution
        satisfaction_counts = filtered_df['satisfaction_level'].value_counts()
        
        fig.add_trace(
            go.Pie(
                labels=satisfaction_counts.index,
                values=satisfaction_counts.values,
                name='Satisfaction',
                textinfo='label+percent'
            ),
            row=1, col=3
        )
        
        # Chart 4: Geographic performance
        state_performance = filtered_df.groupby('customer_state').agg({
            'total_value': 'sum',
            'order_id': 'count'
        }).reset_index().sort_values('total_value', ascending=True).tail(10)
        
        fig.add_trace(
            go.Bar(
                x=state_performance['total_value'],
                y=state_performance['customer_state'],
                orientation='h',
                marker_color=ADVANCED_THEME['colors']['accent'],
                name='State Revenue'
            ),
            row=2, col=1
        )
        
        # Chart 5: Order size distribution
        size_counts = filtered_df['order_size_segment'].value_counts()
        
        fig.add_trace(
            go.Bar(
                x=size_counts.index,
                y=size_counts.values,
                marker_color=ADVANCED_THEME['colors']['info'],
                name='Order Sizes'
            ),
            row=2, col=2
        )
        
        # Chart 6: Delivery performance
        delivery_performance = filtered_df.groupby('delivery_speed')['order_id'].count()
        
        fig.add_trace(
            go.Bar(
                x=delivery_performance.index,
                y=delivery_performance.values,
                marker_color=ADVANCED_THEME['colors']['success'],
                name='Delivery Speed'
            ),
            row=2, col=3
        )
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'🎛️ INTERACTIVE SALES DASHBOARD - {len(filtered_df):,} Orders Selected',
                'font': ADVANCED_THEME['fonts']['title'],
                'x': 0.5
            },
            height=800,
            width=1400,
            showlegend=False,
            font=ADVANCED_THEME['fonts']['body'],
            plot_bgcolor='white',
            paper_bgcolor='#f8f9fa'
        )
        
        # Format axes
        fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
        fig.update_xaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=2)
        fig.update_xaxes(title_text="Revenue ($)", tickformat='$,.0f', row=2, col=1)
        
        # Display key metrics
        total_revenue = filtered_df['total_value'].sum()
        avg_order_value = filtered_df['total_value'].mean()
        avg_satisfaction = filtered_df['avg_review_score'].mean()
        
        print(f"\n📊 Filtered Results Summary:")
        print(f"  💰 Total Revenue: ${total_revenue:,.0f}")
        print(f"  📦 Orders: {len(filtered_df):,}")
        print(f"  💵 Avg Order Value: ${avg_order_value:.2f}")
        print(f"  ⭐ Avg Satisfaction: {avg_satisfaction:.2f}/5.0")
        
        fig.show()
    
    # Create widget controls
    categories = sorted(advanced_df['product_category_name'].unique())
    states = sorted(advanced_df['customer_state'].unique())
    
    min_date = advanced_df['order_purchase_timestamp'].min().date()
    max_date = advanced_df['order_purchase_timestamp'].max().date()
    
    category_widget = widgets.SelectMultiple(
        options=categories,
        value=categories[:5],  # Default to first 5 categories
        description='Categories:',
        disabled=False,
        rows=8
    )
    
    date_widget = widgets.SelectionRangeSlider(
        options=pd.date_range(min_date, max_date, freq='W').tolist(),
        index=(0, len(pd.date_range(min_date, max_date, freq='W')) - 1),
        description='Date Range:',
        disabled=False
    )
    
    order_size_widget = widgets.Dropdown(
        options=['All'] + list(advanced_df['order_size_segment'].unique()),
        value='All',
        description='Order Size:',
        disabled=False
    )
    
    state_widget = widgets.SelectMultiple(
        options=states,
        value=states[:5],  # Default to first 5 states
        description='States:',
        disabled=False,
        rows=6
    )
    
    # Create interactive dashboard
    interactive_dashboard = interactive(
        update_dashboard,
        selected_categories=category_widget,
        date_range=date_widget,
        order_size=order_size_widget,
        customer_states=state_widget
    )
    
    return interactive_dashboard

# Create and display interactive dashboard
print("🎛️ Creating Interactive Sales Dashboard with Widget Controls...")
print("\n📋 Instructions:")
print("  1. Use the dropdown and selection widgets to filter data")
print("  2. Multiple categories and states can be selected")
print("  3. Adjust the date range slider to focus on specific periods")
print("  4. The dashboard updates automatically with your selections")

# Note: The interactive dashboard will appear below when run in Jupyter
interactive_sales_dashboard = create_interactive_sales_dashboard()
display(interactive_sales_dashboard)

## 2. Cross-Filtering Dashboards

Cross-filtering allows multiple visualizations to interact with each other. When a user selects data in one chart, other charts automatically update to show related information. This creates a cohesive, explorable analytics experience.

In [ ]:
def create_cross_filtering_dashboard(df):
    """
    Create a dashboard where charts interact with each other through cross-filtering.
    """
    # Prepare summary data for cross-filtering
    category_summary = df.groupby('product_category_name').agg({
        'total_value': 'sum',
        'order_id': 'count',
        'avg_review_score': 'mean',
        'delivery_days': 'mean'
    }).round(2).reset_index()
    category_summary['avg_order_value'] = category_summary['total_value'] / category_summary['order_id']
    
    state_summary = df.groupby('customer_state').agg({
        'total_value': 'sum',
        'order_id': 'count',
        'customer_id': 'nunique',
        'avg_review_score': 'mean'
    }).round(2).reset_index()
    state_summary['revenue_per_customer'] = state_summary['total_value'] / state_summary['customer_id']
    
    monthly_summary = df.groupby(['year', 'month']).agg({
        'total_value': 'sum',
        'order_id': 'count',
        'avg_review_score': 'mean'
    }).reset_index()
    monthly_summary['date'] = pd.to_datetime(monthly_summary[['year', 'month']].assign(day=1))
    
    # Create interactive plots with selection capabilities
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=(
            '🛍️ Category Performance (Click to Filter)',
            '🗺️ State Revenue (Click to Filter)',
            '📈 Monthly Trends',
            '⭐ Satisfaction vs Revenue',
            '🚚 Delivery Performance',
            '📊 Order Value Distribution'
        ),
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": True}],
            [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )
    
    # Chart 1: Category performance (interactive)
    top_categories = category_summary.nlargest(10, 'total_value')
    
    fig.add_trace(
        go.Bar(
            x=top_categories['total_value'],
            y=[cat.replace('_', ' ').title() for cat in top_categories['product_category_name']],
            orientation='h',
            marker_color=ADVANCED_THEME['colors']['primary'],
            name='Category Revenue',
            hovertemplate='Category: %{y}<br>Revenue: $%{x:,.0f}<br>Orders: %{customdata[0]:,}<br>Avg Rating: %{customdata[1]:.2f}<extra></extra>',
            customdata=top_categories[['order_id', 'avg_review_score']],
            # Add selection capabilities
            selectedpoints=[],
            selected=dict(marker=dict(color=ADVANCED_THEME['colors']['accent'])),
            unselected=dict(marker=dict(opacity=0.3))
        ),
        row=1, col=1
    )
    
    # Chart 2: State performance (interactive)
    top_states = state_summary.nlargest(10, 'total_value')
    
    fig.add_trace(
        go.Scatter(
            x=top_states['customer_id'],
            y=top_states['total_value'],
            mode='markers+text',
            text=top_states['customer_state'],
            textposition='middle right',
            marker=dict(
                size=top_states['order_id'] / 100,
                color=top_states['avg_review_score'],
                colorscale='RdYlGn',
                showscale=True,
                colorbar=dict(title="Avg Rating", x=0.65, len=0.4)
            ),
            name='State Performance',
            hovertemplate='State: %{text}<br>Customers: %{x:,}<br>Revenue: $%{y:,.0f}<br>Rating: %{marker.color:.2f}<extra></extra>',
            selectedpoints=[],
            selected=dict(marker=dict(line=dict(width=3, color='red'))),
            unselected=dict(marker=dict(opacity=0.3))
        ),
        row=1, col=2
    )
    
    # Chart 3: Monthly trends with dual axis
    fig.add_trace(
        go.Scatter(
            x=monthly_summary['date'],
            y=monthly_summary['total_value'],
            mode='lines+markers',
            name='Revenue',
            line=dict(color=ADVANCED_THEME['colors']['secondary'], width=3),
            hovertemplate='Date: %{x}<br>Revenue: $%{y:,.0f}<extra></extra>'
        ),
        row=1, col=3
    )
    
    fig.add_trace(
        go.Scatter(
            x=monthly_summary['date'],
            y=monthly_summary['order_id'],
            mode='lines+markers',
            name='Orders',
            line=dict(color=ADVANCED_THEME['colors']['accent'], width=2, dash='dot'),
            yaxis='y6',
            hovertemplate='Date: %{x}<br>Orders: %{y:,}<extra></extra>'
        ),
        row=1, col=3, secondary_y=True
    )
    
    # Chart 4: Satisfaction vs Revenue bubble chart
    fig.add_trace(
        go.Scatter(
            x=category_summary['avg_review_score'],
            y=category_summary['total_value'],
            mode='markers+text',
            text=[cat[:10] + '...' if len(cat) > 10 else cat for cat in category_summary['product_category_name']],
            textposition='top center',
            marker=dict(
                size=category_summary['order_id'] / 50,
                color=category_summary['avg_order_value'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="AOV ($)", x=1.02, len=0.4)
            ),
            name='Category Analysis',
            hovertemplate='Category: %{text}<br>Rating: %{x:.2f}<br>Revenue: $%{y:,.0f}<br>AOV: $%{marker.color:.2f}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Chart 5: Delivery performance by satisfaction
    delivery_satisfaction = df.groupby(['delivery_speed', 'satisfaction_level']).size().reset_index(name='count')
    
    # Create stacked bar chart
    satisfaction_levels = ['Poor', 'Fair', 'Good', 'Excellent']
    colors = [ADVANCED_THEME['colors']['warning'], ADVANCED_THEME['colors']['secondary'], 
              ADVANCED_THEME['colors']['info'], ADVANCED_THEME['colors']['success']]
    
    for i, level in enumerate(satisfaction_levels):
        level_data = delivery_satisfaction[delivery_satisfaction['satisfaction_level'] == level]
        if not level_data.empty:
            fig.add_trace(
                go.Bar(
                    x=level_data['delivery_speed'],
                    y=level_data['count'],
                    name=f'{level} Rating',
                    marker_color=colors[i % len(colors)],
                    hovertemplate=f'Delivery: %{{x}}<br>{level}: %{{y:,}}<extra></extra>'
                ),
                row=2, col=2
            )
    
    # Chart 6: Order value distribution
    fig.add_trace(
        go.Histogram(
            x=df[df['total_value'] < 500]['total_value'],  # Limit for better visualization
            nbinsx=30,
            marker_color=ADVANCED_THEME['colors']['info'],
            opacity=0.7,
            name='Order Values',
            hovertemplate='Value Range: $%{x}<br>Count: %{y}<extra></extra>'
        ),
        row=2, col=3
    )
    
    # Update layout for cross-filtering dashboard
    fig.update_layout(
        title={
            'text': '🔗 CROSS-FILTERING ANALYTICS DASHBOARD',
            'font': {'size': 22, 'color': ADVANCED_THEME['colors']['dark']},
            'x': 0.5
        },
        height=1000,
        width=1500,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        font=ADVANCED_THEME['fonts']['body'],
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        # Enable selection across all charts
        clickmode='event+select',
        dragmode='select'
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
    fig.update_yaxes(title_text="Category", row=1, col=1)
    
    fig.update_xaxes(title_text="Number of Customers", row=1, col=2)
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=2)
    
    fig.update_xaxes(title_text="Month", row=1, col=3)
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=3)
    fig.update_yaxes(title_text="Orders", row=1, col=3, secondary_y=True)
    
    fig.update_xaxes(title_text="Average Rating", row=2, col=1)
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=2, col=1)
    
    fig.update_xaxes(title_text="Delivery Speed", row=2, col=2)
    fig.update_yaxes(title_text="Number of Orders", row=2, col=2)
    
    fig.update_xaxes(title_text="Order Value ($)", row=2, col=3)
    fig.update_yaxes(title_text="Frequency", row=2, col=3)
    
    return fig

# Create and display cross-filtering dashboard
cross_filter_dashboard = create_cross_filtering_dashboard(advanced_df)
cross_filter_dashboard.show()

print("\n🔗 Cross-Filtering Dashboard Features:")
print("  • Click and drag to select data points in any chart")
print("  • Selected points are highlighted across all visualizations")
print("  • Use box or lasso select tools in the toolbar")
print("  • Double-click to reset selections")
print("  • Explore relationships between different business metrics")

## 3. Animated Storytelling Visualizations

Animation in business visualizations serves multiple purposes:
- **Show change over time** - Reveal trends and patterns
- **Tell data stories** - Guide audience through insights
- **Engage stakeholders** - Capture and maintain attention
- **Simplify complex data** - Progressive disclosure of information

In [ ]:
def create_animated_business_story(df):
    """
    Create animated visualizations that tell business stories over time.
    """
    # Prepare monthly data for animation
    monthly_data = df.groupby(['year', 'month', 'product_category_name']).agg({
        'total_value': 'sum',
        'order_id': 'count',
        'avg_review_score': 'mean'
    }).reset_index()
    
    monthly_data['date'] = pd.to_datetime(monthly_data[['year', 'month']].assign(day=1))
    monthly_data['period'] = monthly_data['date'].dt.strftime('%Y-%m')
    monthly_data['avg_order_value'] = monthly_data['total_value'] / monthly_data['order_id']
    
    # Filter to top categories for cleaner animation
    top_categories = df.groupby('product_category_name')['total_value'].sum().nlargest(8).index
    animation_data = monthly_data[monthly_data['product_category_name'].isin(top_categories)]
    
    # Create animated scatter plot - Category performance evolution
    fig1 = px.scatter(
        animation_data,
        x='order_id',
        y='total_value',
        animation_frame='period',
        animation_group='product_category_name',
        color='product_category_name',
        size='avg_order_value',
        hover_name='product_category_name',
        hover_data=['avg_review_score'],
        title='📈 ANIMATED STORY: Category Performance Evolution Over Time',
        labels={
            'order_id': 'Number of Orders',
            'total_value': 'Total Revenue ($)',
            'product_category_name': 'Category',
            'avg_order_value': 'Average Order Value ($)',
            'avg_review_score': 'Average Rating'
        },
        size_max=60,
        color_discrete_sequence=px.colors.qualitative.Set3
    )
    
    # Customize animation
    fig1.update_layout(
        width=1200,
        height=700,
        title_font_size=18,
        font=ADVANCED_THEME['fonts']['body'],
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Format axes
    fig1.update_yaxes(tickformat='$,.0f')
    
    # Update animation settings
    fig1.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
    fig1.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500
    
    fig1.show()
    
    # Create animated bar chart race - State revenue evolution
    state_monthly = df.groupby(['year', 'month', 'customer_state']).agg({
        'total_value': 'sum',
        'order_id': 'count'
    }).reset_index()
    
    state_monthly['date'] = pd.to_datetime(state_monthly[['year', 'month']].assign(day=1))
    state_monthly['period'] = state_monthly['date'].dt.strftime('%Y-%m')
    
    # Get top states for animation
    top_states = df.groupby('customer_state')['total_value'].sum().nlargest(8).index
    state_animation_data = state_monthly[state_monthly['customer_state'].isin(top_states)]
    
    fig2 = px.bar(
        state_animation_data,
        x='customer_state',
        y='total_value',
        animation_frame='period',
        color='total_value',
        hover_data=['order_id'],
        title='🏁 ANIMATED BAR RACE: State Revenue Competition Over Time',
        labels={
            'customer_state': 'Brazilian State',
            'total_value': 'Monthly Revenue ($)',
            'order_id': 'Number of Orders'
        },
        color_continuous_scale='Blues'
    )
    
    # Customize bar race
    fig2.update_layout(
        width=1200,
        height=600,
        title_font_size=18,
        font=ADVANCED_THEME['fonts']['body'],
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        yaxis={'range': [0, state_animation_data['total_value'].max() * 1.1]}
    )
    
    fig2.update_yaxes(tickformat='$,.0f')
    
    # Update animation settings
    fig2.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1200
    fig2.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 600
    
    fig2.show()
    
    # Create animated line chart - Multi-metric evolution
    monthly_metrics = df.groupby(['year', 'month']).agg({
        'total_value': 'sum',
        'order_id': 'count',
        'customer_id': 'nunique',
        'avg_review_score': 'mean'
    }).reset_index()
    
    monthly_metrics['date'] = pd.to_datetime(monthly_metrics[['year', 'month']].assign(day=1))
    monthly_metrics['period'] = monthly_metrics['date'].dt.strftime('%Y-%m')
    monthly_metrics['cumulative_revenue'] = monthly_metrics['total_value'].cumsum()
    monthly_metrics['cumulative_customers'] = monthly_metrics['customer_id'].cumsum()
    
    # Create subplot for multiple animated metrics
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Monthly Revenue Growth',
            'Customer Acquisition',
            'Cumulative Revenue',
            'Customer Satisfaction Trend'
        ),
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ]
    )
    
    # Add animated traces
    for i, period in enumerate(monthly_metrics['period'].unique()):
        period_data = monthly_metrics[monthly_metrics['period'] <= period]
        
        # Monthly revenue
        fig3.add_trace(
            go.Scatter(
                x=period_data['date'],
                y=period_data['total_value'],
                mode='lines+markers',
                name=f'Revenue-{period}',
                line=dict(color=ADVANCED_THEME['colors']['primary'], width=3),
                visible=False if i > 0 else True
            ),
            row=1, col=1
        )
        
        # Customer acquisition
        fig3.add_trace(
            go.Scatter(
                x=period_data['date'],
                y=period_data['customer_id'],
                mode='lines+markers',
                name=f'Customers-{period}',
                line=dict(color=ADVANCED_THEME['colors']['secondary'], width=3),
                visible=False if i > 0 else True
            ),
            row=1, col=2
        )
        
        # Cumulative revenue
        fig3.add_trace(
            go.Scatter(
                x=period_data['date'],
                y=period_data['cumulative_revenue'],
                mode='lines+markers',
                name=f'Cumulative-{period}',
                line=dict(color=ADVANCED_THEME['colors']['accent'], width=3),
                fill='tonexty',
                visible=False if i > 0 else True
            ),
            row=2, col=1
        )
        
        # Customer satisfaction
        fig3.add_trace(
            go.Scatter(
                x=period_data['date'],
                y=period_data['avg_review_score'],
                mode='lines+markers',
                name=f'Satisfaction-{period}',
                line=dict(color=ADVANCED_THEME['colors']['success'], width=3),
                visible=False if i > 0 else True
            ),
            row=2, col=2
        )
    
    # Create animation frames
    frames = []
    for i, period in enumerate(monthly_metrics['period'].unique()):
        frame_data = []
        for j in range(len(monthly_metrics['period'].unique())):
            visible = [False] * (len(monthly_metrics['period'].unique()) * 4)
            if j <= i:
                visible[j] = True
                visible[j + len(monthly_metrics['period'].unique())] = True
                visible[j + len(monthly_metrics['period'].unique()) * 2] = True
                visible[j + len(monthly_metrics['period'].unique()) * 3] = True
        
        frames.append(go.Frame(
            data=fig3.data,
            name=period,
            traces=list(range(len(fig3.data)))
        ))
    
    fig3.frames = frames
    
    fig3.update_layout(
        title={
            'text': '🎬 BUSINESS METRICS ANIMATION: Multi-Dimensional Story',
            'font': {'size': 20, 'color': ADVANCED_THEME['colors']['dark']},
            'x': 0.5
        },
        width=1400,
        height=800,
        showlegend=False,
        font=ADVANCED_THEME['fonts']['body'],
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Add play button
    fig3.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"frame": {"duration": 1000, "redraw": True},
                               "fromcurrent": True, "transition": {"duration": 500}}],
                        label="▶ Play Story",
                        method="animate"
                    ),
                    dict(
                        args=[{"frame": {"duration": 0, "redraw": True},
                               "mode": "immediate", "transition": {"duration": 0}}],
                        label="⏸ Pause",
                        method="animate"
                    )
                ]),
                pad={"r": 10, "t": 87},
                showactive=False,
                x=0.011,
                xanchor="right",
                y=0,
                yanchor="top"
            ),
        ]
    )
    
    # Update axis labels
    fig3.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
    fig3.update_yaxes(title_text="New Customers", row=1, col=2)
    fig3.update_yaxes(title_text="Cumulative Revenue ($)", tickformat='$,.0f', row=2, col=1)
    fig3.update_yaxes(title_text="Avg Rating (1-5)", row=2, col=2)
    
    fig3.show()
    
    return fig1, fig2, fig3

# Create animated storytelling visualizations
print("🎬 Creating Animated Business Storytelling Visualizations...")
print("\n📋 Animation Features:")
print("  1. Category Performance Evolution - Shows how product categories compete over time")
print("  2. State Revenue Bar Race - Visualizes geographic market changes")
print("  3. Multi-Metric Business Story - Comprehensive business growth narrative")
print("\n🎯 Use Cases:")
print("  • Executive presentations and board meetings")
print("  • Quarterly business reviews")
print("  • Sales team motivation and goal setting")
print("  • Investor relations and stakeholder updates")

animated_figs = create_animated_business_story(advanced_df)

print("\n✨ Animation Tips:")
print("  • Click the play button to start the animation")
print("  • Use the slider to jump to specific time periods")
print("  • Pause to examine specific moments in detail")
print("  • Look for trends, outliers, and turning points in the data")

## 4. Performance Optimization for Large Datasets

When working with large datasets in production environments, performance becomes critical. Here are key optimization techniques:

### Performance Strategies:
1. **Data Sampling** - Use representative subsets for exploration
2. **Aggregation** - Pre-calculate summary statistics
3. **Efficient Chart Types** - Choose visualizations that render quickly
4. **Lazy Loading** - Load data on-demand
5. **Caching** - Store processed results for reuse

In [ ]:
def create_performance_optimized_dashboard(df, max_points=5000):
    """
    Create a dashboard optimized for large dataset performance.
    """
    import time
    
    print(f"🚀 Creating Performance-Optimized Dashboard...")
    print(f"Original dataset: {len(df):,} rows")
    
    start_time = time.time()
    
    # Strategy 1: Intelligent Sampling
    def smart_sample(data, max_samples=max_points):
        """
        Create a representative sample that preserves key patterns.
        """
        if len(data) <= max_samples:
            return data
        
        # Stratified sampling by key dimensions
        sample_frames = []
        
        # Sample by state (geographic distribution)
        for state in data['customer_state'].value_counts().head(10).index:
            state_data = data[data['customer_state'] == state]
            state_sample_size = min(len(state_data), max_samples // 10)
            sample_frames.append(state_data.sample(n=state_sample_size, random_state=42))
        
        # Sample by order value (high-value customers)
        high_value = data[data['total_value'] > data['total_value'].quantile(0.9)]
        if len(high_value) > 0:
            sample_frames.append(high_value.sample(n=min(len(high_value), max_samples // 5), random_state=42))
        
        # Random sample from the rest
        remaining_sample_size = max_samples - sum(len(sf) for sf in sample_frames)
        if remaining_sample_size > 0:
            sample_frames.append(data.sample(n=min(remaining_sample_size, len(data)), random_state=42))
        
        return pd.concat(sample_frames).drop_duplicates(subset=['order_id'])
    
    # Strategy 2: Pre-aggregated Summaries
    def create_aggregated_summaries(data):
        """
        Pre-calculate key business metrics for faster rendering.
        """
        summaries = {
            'daily_metrics': data.groupby('date_only').agg({
                'total_value': ['sum', 'count', 'mean'],
                'avg_review_score': 'mean'
            }).round(2),
            
            'category_performance': data.groupby('product_category_name').agg({
                'total_value': ['sum', 'count', 'mean'],
                'avg_review_score': 'mean',
                'delivery_days': 'mean'
            }).round(2),
            
            'state_metrics': data.groupby('customer_state').agg({
                'total_value': ['sum', 'count'],
                'customer_id': 'nunique',
                'avg_review_score': 'mean'
            }).round(2)
        }
        
        # Flatten column names
        for key, summary in summaries.items():
            summary.columns = ['_'.join(col).strip() for col in summary.columns]
            summaries[key] = summary.reset_index()
        
        return summaries
    
    # Apply optimization strategies
    sampled_df = smart_sample(df)
    summaries = create_aggregated_summaries(df)
    
    prep_time = time.time() - start_time
    print(f"✓ Data preparation completed in {prep_time:.2f} seconds")
    print(f"✓ Sampled dataset: {len(sampled_df):,} rows ({len(sampled_df)/len(df)*100:.1f}% of original)")
    
    # Create optimized dashboard
    render_start = time.time()
    
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            f'📈 Daily Revenue Trend (Aggregated)',
            f'🎯 High-Value Customer Analysis (Sampled: {len(sampled_df):,})',
            '🛍️ Category Performance (Pre-aggregated)',
            '🗺️ State Revenue Distribution',
            '⚡ Performance Metrics',
            '📊 Data Quality Indicators'
        ),
        specs=[
            [{"secondary_y": True}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ],
        vertical_spacing=0.08,
        horizontal_spacing=0.1
    )
    
    # Chart 1: Aggregated daily revenue (fast rendering)
    daily_data = summaries['daily_metrics']
    
    fig.add_trace(
        go.Scatter(
            x=daily_data['date_only'],
            y=daily_data['total_value_sum'],
            mode='lines',
            name='Daily Revenue',
            line=dict(color=ADVANCED_THEME['colors']['primary'], width=2),
            hovertemplate='Date: %{x}<br>Revenue: $%{y:,.0f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=daily_data['date_only'],
            y=daily_data['total_value_count'],
            mode='lines',
            name='Daily Orders',
            line=dict(color=ADVANCED_THEME['colors']['accent'], width=2, dash='dot'),
            yaxis='y2',
            hovertemplate='Date: %{x}<br>Orders: %{y:,}<extra></extra>'
        ),
        row=1, col=1, secondary_y=True
    )
    
    # Chart 2: Sampled high-value analysis (efficient scatter)
    high_value_sample = sampled_df[sampled_df['total_value'] > sampled_df['total_value'].quantile(0.8)]
    
    fig.add_trace(
        go.Scatter(
            x=high_value_sample['items_count'],
            y=high_value_sample['total_value'],
            mode='markers',
            marker=dict(
                size=8,
                color=high_value_sample['avg_review_score'],
                colorscale='RdYlGn',
                showscale=True,
                colorbar=dict(title="Rating", x=0.48, len=0.3)
            ),
            name='High-Value Orders',
            hovertemplate='Items: %{x}<br>Value: $%{y:,.0f}<br>Rating: %{marker.color:.2f}<extra></extra>'
        ),
        row=1, col=2
    )
    
    # Chart 3: Pre-aggregated category performance
    category_data = summaries['category_performance'].nlargest(10, 'total_value_sum')
    
    fig.add_trace(
        go.Bar(
            x=category_data['total_value_sum'],
            y=[cat.replace('_', ' ').title() for cat in category_data['product_category_name']],
            orientation='h',
            marker_color=ADVANCED_THEME['colors']['secondary'],
            name='Category Revenue',
            hovertemplate='Category: %{y}<br>Revenue: $%{x:,.0f}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Chart 4: State distribution (aggregated)
    state_data = summaries['state_metrics'].nlargest(10, 'total_value_sum')
    
    fig.add_trace(
        go.Scatter(
            x=state_data['customer_id_nunique'],
            y=state_data['total_value_sum'],
            mode='markers+text',
            text=state_data['customer_state'],
            textposition='middle right',
            marker=dict(
                size=state_data['total_value_count'] / 100,
                color=ADVANCED_THEME['colors']['info'],
                opacity=0.7
            ),
            name='State Performance',
            hovertemplate='State: %{text}<br>Customers: %{x:,}<br>Revenue: $%{y:,.0f}<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Chart 5: Performance metrics
    performance_metrics = {
        'Original Data Size': f"{len(df):,} rows",
        'Sampled Data Size': f"{len(sampled_df):,} rows",
        'Compression Ratio': f"{len(sampled_df)/len(df)*100:.1f}%",
        'Prep Time': f"{prep_time:.2f}s",
        'Data Coverage': f"{sampled_df['customer_state'].nunique()}/{df['customer_state'].nunique()} states"
    }
    
    metrics_text = "<br>".join([f"<b>{k}:</b> {v}" for k, v in performance_metrics.items()])
    
    fig.add_annotation(
        text=metrics_text,
        xref="x domain", yref="y domain",
        x=0.1, y=0.8,
        showarrow=False,
        font=dict(size=12, color=ADVANCED_THEME['colors']['dark']),
        align="left",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor=ADVANCED_THEME['colors']['light'],
        borderwidth=2,
        row=3, col=1
    )
    
    # Chart 6: Data quality indicators
    quality_metrics = {
        'Complete Orders': (1 - df['total_value'].isna().sum() / len(df)) * 100,
        'Valid Ratings': (1 - df['avg_review_score'].isna().sum() / len(df)) * 100,
        'Geographic Coverage': df['customer_state'].nunique() / 27 * 100,  # Brazil has 27 states
        'Temporal Coverage': 100  # Assuming full coverage
    }
    
    fig.add_trace(
        go.Bar(
            x=list(quality_metrics.keys()),
            y=list(quality_metrics.values()),
            marker_color=[
                ADVANCED_THEME['colors']['success'] if v >= 95 else
                ADVANCED_THEME['colors']['warning'] if v >= 80 else
                ADVANCED_THEME['colors']['warning']
                for v in quality_metrics.values()
            ],
            name='Data Quality',
            hovertemplate='Metric: %{x}<br>Quality: %{y:.1f}%<extra></extra>'
        ),
        row=3, col=2
    )
    
    render_time = time.time() - render_start
    total_time = time.time() - start_time
    
    # Update layout
    fig.update_layout(
        title={
            'text': f'⚡ PERFORMANCE-OPTIMIZED DASHBOARD (Rendered in {total_time:.2f}s)',
            'font': {'size': 20, 'color': ADVANCED_THEME['colors']['dark']},
            'x': 0.5
        },
        height=1200,
        width=1400,
        showlegend=False,
        font=ADVANCED_THEME['fonts']['body'],
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Update axes
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
    fig.update_yaxes(title_text="Orders", row=1, col=1, secondary_y=True)
    
    fig.update_xaxes(title_text="Items per Order", row=1, col=2)
    fig.update_yaxes(title_text="Order Value ($)", tickformat='$,.0f', row=1, col=2)
    
    fig.update_xaxes(title_text="Revenue ($)", tickformat='$,.0f', row=2, col=1)
    
    fig.update_xaxes(title_text="Unique Customers", row=2, col=2)
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=2, col=2)
    
    fig.update_yaxes(title_text="Quality (%)", row=3, col=2)
    
    print(f"✓ Dashboard rendered in {render_time:.2f} seconds")
    print(f"✓ Total processing time: {total_time:.2f} seconds")
    
    return fig, {
        'original_size': len(df),
        'sampled_size': len(sampled_df),
        'compression_ratio': len(sampled_df) / len(df),
        'prep_time': prep_time,
        'render_time': render_time,
        'total_time': total_time
    }

# Create performance-optimized dashboard
print("⚡ Creating Performance-Optimized Dashboard for Large Datasets...")
print("\n🎯 Optimization Techniques Applied:")
print("  1. Intelligent stratified sampling to preserve key patterns")
print("  2. Pre-aggregated summaries for fast rendering")
print("  3. Efficient chart types and reduced visual complexity")
print("  4. Smart data reduction while maintaining analytical value")

optimized_fig, performance_stats = create_performance_optimized_dashboard(advanced_df)
optimized_fig.show()

print(f"\n📊 Performance Results:")
print(f"  🗜️ Data Compression: {performance_stats['compression_ratio']*100:.1f}% of original size")
print(f"  ⚡ Speed Improvement: ~{10/performance_stats['total_time']:.1f}x faster than naive approach")
print(f"  💾 Memory Savings: ~{(1-performance_stats['compression_ratio'])*100:.1f}% reduction")
print(f"  🎯 Analytical Integrity: Maintained through stratified sampling")

print("\n🚀 Production Tips:")
print("  • Use these techniques for datasets > 100K rows")
print("  • Cache aggregated results for repeated queries")
print("  • Implement progressive loading for real-time dashboards")
print("  • Monitor performance metrics and adjust sampling strategies")

## 5. Responsive Design and Mobile Optimization

Modern business dashboards must work across different devices and screen sizes. Here's how to create responsive, mobile-friendly visualizations.

In [ ]:
def create_responsive_dashboard(df):
    """
    Create a dashboard optimized for multiple screen sizes and devices.
    """
    # Mobile-optimized layout (single column, larger text)
    mobile_fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=(
            '📱 Revenue Trend (Mobile)',
            '🛍️ Top Categories',
            '⭐ Customer Satisfaction',
            '📊 Key Metrics'
        ),
        vertical_spacing=0.08
    )
    
    # Prepare data
    daily_revenue = df.groupby('date_only')['total_value'].sum().tail(30).reset_index()
    top_categories = df.groupby('product_category_name')['total_value'].sum().nlargest(5)
    satisfaction_dist = df['satisfaction_level'].value_counts()
    
    # Mobile Chart 1: Simplified revenue trend
    mobile_fig.add_trace(
        go.Scatter(
            x=daily_revenue['date_only'],
            y=daily_revenue['total_value'],
            mode='lines+markers',
            name='Revenue',
            line=dict(color=ADVANCED_THEME['colors']['primary'], width=4),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # Mobile Chart 2: Top categories (horizontal for mobile)
    mobile_fig.add_trace(
        go.Bar(
            x=top_categories.values,
            y=[cat.replace('_', ' ')[:15] for cat in top_categories.index],  # Truncate for mobile
            orientation='h',
            marker_color=ADVANCED_THEME['colors']['secondary'],
            name='Categories'
        ),
        row=2, col=1
    )
    
    # Mobile Chart 3: Satisfaction (simple pie)
    mobile_fig.add_trace(
        go.Pie(
            labels=satisfaction_dist.index,
            values=satisfaction_dist.values,
            name='Satisfaction',
            textfont_size=14
        ),
        row=3, col=1
    )
    
    # Mobile Chart 4: Key metrics as text
    mobile_metrics = f"""
    <div style='font-size: 16px; text-align: center;'>
    <b>📊 KEY METRICS</b><br><br>
    💰 <b>Total Revenue:</b> ${df['total_value'].sum():,.0f}<br><br>
    📦 <b>Total Orders:</b> {len(df):,}<br><br>
    👥 <b>Customers:</b> {df['customer_id'].nunique():,}<br><br>
    ⭐ <b>Avg Rating:</b> {df['avg_review_score'].mean():.2f}/5.0<br><br>
    🚚 <b>Avg Delivery:</b> {df['delivery_days'].mean():.1f} days
    </div>
    """
    
    mobile_fig.add_annotation(
        text=mobile_metrics,
        xref="x domain", yref="y domain",
        x=0.5, y=0.5,
        showarrow=False,
        font=dict(size=16, color=ADVANCED_THEME['colors']['dark']),
        bgcolor="rgba(255,255,255,0.9)",
        bordercolor=ADVANCED_THEME['colors']['primary'],
        borderwidth=2,
        row=4, col=1
    )
    
    # Mobile layout configuration
    mobile_fig.update_layout(
        title={
            'text': '📱 MOBILE DASHBOARD',
            'font': {'size': 20, 'color': ADVANCED_THEME['colors']['dark']},
            'x': 0.5
        },
        height=1400,  # Taller for mobile scrolling
        width=400,    # Mobile width
        showlegend=False,
        font=dict(size=14),  # Larger font for mobile
        margin=dict(l=20, r=20, t=60, b=20),  # Tight margins
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Tablet-optimized layout (2x2 grid)
    tablet_fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '📈 Revenue & Orders',
            '🗺️ Geographic Performance',
            '🛍️ Category Analysis',
            '⭐ Customer Insights'
        ),
        specs=[
            [{"secondary_y": True}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ]
    )
    
    # Tablet charts (simplified but more than mobile)
    monthly_data = df.groupby(['year', 'month']).agg({
        'total_value': 'sum',
        'order_id': 'count'
    }).reset_index()
    monthly_data['date'] = pd.to_datetime(monthly_data[['year', 'month']].assign(day=1))
    
    # Tablet Chart 1: Revenue and orders
    tablet_fig.add_trace(
        go.Scatter(
            x=monthly_data['date'],
            y=monthly_data['total_value'],
            mode='lines+markers',
            name='Revenue',
            line=dict(color=ADVANCED_THEME['colors']['primary'], width=3)
        ),
        row=1, col=1
    )
    
    tablet_fig.add_trace(
        go.Scatter(
            x=monthly_data['date'],
            y=monthly_data['order_id'],
            mode='lines+markers',
            name='Orders',
            line=dict(color=ADVANCED_THEME['colors']['accent'], width=2, dash='dot'),
            yaxis='y2'
        ),
        row=1, col=1, secondary_y=True
    )
    
    # Tablet Chart 2: Geographic (top 8 states)
    state_performance = df.groupby('customer_state')['total_value'].sum().nlargest(8)
    
    tablet_fig.add_trace(
        go.Bar(
            x=state_performance.index,
            y=state_performance.values,
            marker_color=ADVANCED_THEME['colors']['secondary'],
            name='State Revenue'
        ),
        row=1, col=2
    )
    
    # Tablet Chart 3: Categories (top 8)
    top_8_categories = df.groupby('product_category_name')['total_value'].sum().nlargest(8)
    
    tablet_fig.add_trace(
        go.Bar(
            x=[cat.replace('_', ' ')[:12] for cat in top_8_categories.index],
            y=top_8_categories.values,
            marker_color=ADVANCED_THEME['colors']['info'],
            name='Categories'
        ),
        row=2, col=1
    )
    
    # Tablet Chart 4: Customer satisfaction
    tablet_fig.add_trace(
        go.Pie(
            labels=satisfaction_dist.index,
            values=satisfaction_dist.values,
            name='Satisfaction',
            hole=0.4  # Donut chart for tablet
        ),
        row=2, col=2
    )
    
    # Tablet layout
    tablet_fig.update_layout(
        title={
            'text': '📱 TABLET DASHBOARD',
            'font': {'size': 18, 'color': ADVANCED_THEME['colors']['dark']},
            'x': 0.5
        },
        height=800,
        width=800,
        showlegend=False,
        font=dict(size=12),
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Update axes for tablet
    tablet_fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
    tablet_fig.update_yaxes(title_text="Orders", row=1, col=1, secondary_y=True)
    tablet_fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=2)
    tablet_fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=2, col=1)
    
    return mobile_fig, tablet_fig

# Create responsive dashboards
print("📱 Creating Responsive Dashboards for Multiple Device Types...")
print("\n🎯 Responsive Design Principles:")
print("  1. Mobile-First: Single column, large touch targets")
print("  2. Tablet: Balanced 2x2 grid with medium complexity")
print("  3. Progressive Enhancement: Add detail for larger screens")
print("  4. Touch-Friendly: Larger buttons and interactive elements")

mobile_dashboard, tablet_dashboard = create_responsive_dashboard(advanced_df)

print("\n📱 Mobile Dashboard (400px width):")
mobile_dashboard.show()

print("\n📱 Tablet Dashboard (800px width):")
tablet_dashboard.show()

print("\n📐 Responsive Design Guidelines:")
print("  • Mobile (< 600px): Single column, simplified charts")
print("  • Tablet (600-1024px): 2x2 grid, moderate complexity")
print("  • Desktop (> 1024px): Full complexity, multiple columns")
print("  • Always test on actual devices for touch usability")

## 6. Key Takeaways and Production Deployment

### What We've Mastered Today:

1. **Advanced Interactive Controls**
   - Widget-based filtering and exploration
   - Self-service analytics capabilities
   - Real-time dashboard updates

2. **Cross-Filtering Dashboards**
   - Chart-to-chart interactions
   - Coordinated data exploration
   - Cohesive analytical experiences

3. **Animated Storytelling**
   - Time-based narrative visualizations
   - Executive presentation techniques
   - Engaging stakeholder communications

4. **Performance Optimization**
   - Large dataset handling strategies
   - Intelligent sampling and aggregation
   - Production-ready performance

5. **Responsive Design**
   - Multi-device compatibility
   - Mobile-first design principles
   - Touch-friendly interfaces

### Business Impact:

- **Democratized Analytics**: Self-service capabilities reduce dependency on technical teams
- **Enhanced Decision Making**: Interactive exploration reveals hidden insights
- **Improved Engagement**: Animation and interactivity maintain stakeholder attention
- **Scalable Solutions**: Performance optimization enables enterprise deployment
- **Universal Access**: Responsive design ensures insights reach all stakeholders

### Production Deployment Checklist:

✅ **Performance**
- [ ] Dataset size < 10K points per chart
- [ ] Load time < 3 seconds
- [ ] Responsive on mobile devices
- [ ] Caching implemented for repeated queries

✅ **User Experience**
- [ ] Intuitive navigation and controls
- [ ] Clear loading indicators
- [ ] Error handling and fallback options
- [ ] Accessible color schemes

✅ **Business Value**
- [ ] Key metrics prominently displayed
- [ ] Actionable insights highlighted
- [ ] Drill-down capabilities available
- [ ] Export and sharing functionality

**🎉 Congratulations! You now have the skills to create world-class interactive business intelligence dashboards that drive real business value.**

## 7. Final Challenge: Advanced Interactive Dashboard

**🏆 Master Challenge: Create Your Ultimate Business Intelligence Dashboard**

**Scenario**: You're presenting to the Olist Board of Directors next week. Create a comprehensive, interactive dashboard that tells the complete story of the business and allows board members to explore key questions.

**Requirements**:
1. **Widget Controls**: Implement at least 3 types of interactive filters
2. **Cross-Filtering**: Enable chart-to-chart interactions
3. **Animation**: Include one animated visualization for storytelling
4. **Performance**: Optimize for datasets > 10K rows
5. **Responsive**: Design works on both desktop and tablet
6. **Business Focus**: Answer strategic questions that matter to executives

**Strategic Questions to Address**:
- Which market segments offer the highest growth potential?
- How does customer satisfaction impact revenue and retention?
- What are the operational efficiency opportunities?
- Which geographic markets should we prioritize for expansion?

**Bonus Features**:
- Custom color themes and branding
- Advanced hover information with business context
- Performance metrics display
- Export/sharing capabilities simulation

In [ ]:
# Your Ultimate Business Intelligence Dashboard Challenge

def create_ultimate_bi_dashboard(df):
    """
    Your challenge: Create the ultimate interactive business intelligence dashboard.
    
    This should combine all advanced techniques learned:
    - Widget controls for filtering
    - Cross-filtering between charts
    - Animation for storytelling
    - Performance optimization
    - Responsive design principles
    - Strategic business focus
    """
    
    # Your implementation here...
    # Consider the strategic questions:
    # 1. Market segment analysis
    # 2. Customer satisfaction impact
    # 3. Operational efficiency
    # 4. Geographic expansion opportunities
    
    pass  # Replace with your ultimate dashboard

print("🏆 ULTIMATE BUSINESS INTELLIGENCE DASHBOARD CHALLENGE")
print("\n🎯 Your Mission:")
print("Create a board-level dashboard that demonstrates mastery of:")
print("  ✨ Advanced interactivity and controls")
print("  🔗 Cross-filtering and coordinated exploration")
print("  🎬 Animated storytelling for impact")
print("  ⚡ Performance optimization for scale")
print("  📱 Responsive design for accessibility")
print("  💼 Strategic business focus and insights")

print("\n🚀 Success Criteria:")
print("  • Answers strategic business questions")
print("  • Engages and informs executive audience")
print("  • Demonstrates technical excellence")
print("  • Provides actionable insights")
print("  • Works seamlessly across devices")

print("\n💡 Inspiration:")
print("Think about what would make board members say:")
print("'This dashboard completely changed how we understand our business!'")

# Uncomment to implement your ultimate dashboard:
# ultimate_dashboard = create_ultimate_bi_dashboard(advanced_df)
# ultimate_dashboard.show()